In [ ]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time

import warnings
warnings.filterwarnings("ignore")


from IPython.display import display
from tqdm import tqdm_notebook
import gc, psutil

print(l5kit.__version__)

In [ ]:
# Memory measurement
def memory(verbose=True):
    mem = psutil.virtual_memory()
    gb = 1024*1024*1024
    if verbose:
        print('Physical memory:',
              '%.2f GB (used),'%((mem.total - mem.available) / gb),
              '%.2f GB (available)'%((mem.available) / gb), '/',
              '%.2f GB'%(mem.total / gb))
    return (mem.total - mem.available) / gb

def gc_memory(verbose=True):
    m = gc.collect()
    if verbose:
        print('GC:', m, end=' | ')
        memory()

memory();

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed(42)

In [ ]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': '/kaggle/input/lyft-motion-prediction-autonomous-vehicles',
    'model_params': {
        'first_layer_bias': False,
        'pretrained': True,
        'multi_mode': True,
        'model_architecture': 'resnet101',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "model_resnet34_output",
        'lr': 1e-5,
        'weight_path': '../input/resnet101-best-weights-epoch2/model_state_280000.pth',
        'train': False,
        'predict': True,
    },
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
    },
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4,
    },    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4,
    },
    'train_params': {
#         'steps': 100,
#         'update_steps': 10,
#         'checkpoint_steps': 50,
        'steps': 1000,
        'update_steps': 100,
        'checkpoint_steps': 1000,
    },
    
    'combine': True
    
}

# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager()



# Build rasterizer
rasterizer = build_rasterizer(cfg, dm)


# Test dataset
test_cfg = cfg["test_data_loader"]
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open(cached=False)  # to prevent run out of memory
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset, shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"], num_workers=test_cfg["num_workers"])
print(test_dataset)



# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)


import torch
import torchvision

from torch import nn
from torchvision.models.resnet import resnet50, resnet101, resnet34
from typing import Dict

class LyftMultiModel(nn.Module):
    def __init__(self, conf: Dict):
        super().__init__()
        
        self.future_num_frames = cfg["model_params"]["future_num_frames"]
        target_count = 2 * self.future_num_frames

        if cfg["model_params"]["multi_mode"]:
            self.multi_mode = True
            target_count += 1   # One confidence per prediction
            target_count *= 3   # 3 predictions instead of 1
        else:
            self.multi_mode = False

        history_channel_count = (conf["model_params"]["history_num_frames"] + 1) * 2
        total_channel_count = 3 + history_channel_count

        architecture = cfg["model_params"]["model_architecture"]

        if architecture == "resnet50":
            backbone = resnet50(pretrained=cfg["model_params"]["pretrained"])  
        elif architecture == "resnet101":
            backbone = resnet101(pretrained=cfg["model_params"]["pretrained"])
        elif architecture == "resnet34":
            backbone = resnet34(pretrained=cfg["model_params"]["pretrained"])
            # architecture = conf["model_params"]["model_architecture"]
            # backbone = eval(architecture)(pretrained=True)

        if architecture in ["resnet50", "resnet101", "resnet34"]:
            backbone.conv1 = nn.Conv2d(
                total_channel_count,
                backbone.conv1.out_channels,
                kernel_size=backbone.conv1.kernel_size,
                stride=backbone.conv1.stride,
                padding=backbone.conv1.padding,
                bias=cfg["model_params"]["first_layer_bias"], # Maybe True is better?
            )

            if architecture == "resnet34":
                backbone.fc = nn.Linear(in_features=512, out_features=target_count)
            else:
                backbone.fc = nn.Linear(in_features=2048, out_features=target_count)

            self.backbone = backbone

    def forward(self, x):
        y = self.backbone(x)

        if self.multi_mode:
            batches, _ = y.shape

            # print("y.shape =", y.shape)
            # print("batches =", batches)

            pred, confidences = torch.split(y, self.future_num_frames * 3 * 2, dim=1)
            pred = pred.view(batches, 3, self.future_num_frames, 2)

            # print("confidences.shape = ", confidences.shape)

            # assert confidences.shape == (batches, 3)

            confidences = torch.softmax(confidences, dim=1)

            return pred, confidences
        else:
            return y
    
    
def forward(data, model, device, criterion=pytorch_neg_multi_log_likelihood_batch, compute_loss=True):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    preds, confidences = model(inputs)
    # skip compute loss if we are doing prediction
    loss = criterion(targets, preds, confidences, target_availabilities) if compute_loss else 0
    return loss, preds, confidences




def execute(path):
    # ==== INIT MODEL=================
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = LyftMultiModel(cfg)

    #load weight if there is a pretrained model
    weight_path = cfg["model_params"]["weight_path"]
    if weight_path:
        model.load_state_dict(torch.load(weight_path))

    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=cfg["model_params"]["lr"])
    print(f'device {device}')




    if cfg["model_params"]["predict"]:

        model.eval()
        torch.set_grad_enabled(False)

        # store information for evaluation
        future_coords_offsets_pd = []
        timestamps = []
        confidences_list = []
        agent_ids = []
        memorys_pred = []
        t0 = time.time()
        times_pred = []
        iterations_pred = []

        for i, data in enumerate(tqdm_notebook(test_dataloader, mininterval=5.)):

            _, preds, confidences = forward(data, model, device, compute_loss=False)

            preds = torch.einsum('bmti,bji->bmtj', 
                                 preds.double(), 
                                 data["world_from_agent"].to(device)[:, :2, :2]).cpu().numpy()

            future_coords_offsets_pd.append(preds.copy())
            confidences_list.append(confidences.cpu().numpy().copy())
            timestamps.append(data["timestamp"].numpy().copy())
            agent_ids.append(data["track_id"].numpy().copy()) 

            if i%50 == 0:
                t = ((time.time() - t0) / 60)
                print('%4d'%i, '%6.2fmins'%t, end=' | ')
                mem = memory()
                iterations_pred.append(i)
                memorys_pred.append(mem)
                times_pred.append(t)
    #             if i > 0:
    #                 break
        print('Total timespent: %6.2fmins'%((time.time() - t0) / 60))
        memory()


    # create submission to submit to Kaggle
    pred_path = path
    write_pred_csv(
        pred_path,
        timestamps=np.concatenate(timestamps),
        track_ids=np.concatenate(agent_ids),
        coords=np.concatenate(future_coords_offsets_pd),
        confs=np.concatenate(confidences_list),
    )


    df_sub = pd.read_csv(pred_path)
    display(df_sub)

In [ ]:
if cfg['combine']:

    execute('submission_1.csv')

    cfg["model_params"]["weight_path"] = '../input/resnet34-bestscore-epoch-weights/model_state_last.pth'
    cfg['model_params']['model_architecture'] = 'resnet34'

    execute('submission_2.csv')
    
else:
    execute('submission.csv')



In [ ]:
import pandas as pd

if cfg['combine']:
    pd.options.display.max_columns=305

    paths = [
        "submission_1.csv", 
        "submission_2.csv",
    ]

    weights = [0.2, 0.8]

    conf_cols = np.array(["conf_0", "conf_1", "conf_2"])


    xy_cols = [[],[],[]]
    for i in range(50):
        for j in range(3):
            xy_cols[j].append(f"coord_x{j}{i}")
            xy_cols[j].append(f"coord_y{j}{i}")
    xy_cols[0][:10]


    COLUMNS = ["timestamp", "track_id"] + list(conf_cols) + xy_cols[0] + xy_cols[1] + xy_cols[2]

    def sort_df(df, sort_timestamp_track_id=True):

        conf_orders = np.argsort(-df[conf_cols].values,1)
        XY = np.stack([df[xy_cols[0]].values,df[xy_cols[1]].values, df[xy_cols[2]].values], axis=1)
        XY = XY[np.arange(len(XY))[:, None], conf_orders]

        df2 = pd.DataFrame(columns = COLUMNS)
        df2["timestamp"] = df["timestamp"].values
        df2["track_id"] = df["track_id"].values
        df2[xy_cols[0] + xy_cols[1] + xy_cols[2]] = XY.reshape(-1,300)
        df2[conf_cols] = df[conf_cols].values[np.arange(len(df))[:, None], conf_orders]

        if sort_timestamp_track_id:
            df2.sort_values(["timestamp", "track_id"], inplace=True)
            df2.reset_index(inplace=True, drop=True)
        return df2

    df = None
    for path,w in zip(paths,weights):
        print(w, path)
        temp = pd.read_csv(path)
        temp = sort_df(temp)
        temp[COLUMNS[5:]] *= w
        if df is None:
            df = temp
        else:
            df[COLUMNS[2:]] += temp[COLUMNS[2:]]
    df[conf_cols] /= df[conf_cols].sum(1).values[:, None]

    sample = pd.read_csv("../input/lyft-motion-prediction-autonomous-vehicles/multi_mode_sample_submission.csv")

    df = sample[["timestamp", "track_id"]].merge(df, on=["timestamp", "track_id"])
    sample.shape, df.shape

    df.to_csv("submission.csv", index=False, float_format='%.6f')